In [1]:
pip install langchain langchain-community langchainhub chromadb pypdf groq pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.8 MB/s eta 0:

In [1]:
pip install --upgrade langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 25.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.71
    Uninstalling langchain-core-0.3.71:
      Successfully uninstalled langchain-core-0.3.71
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.26
    Uninstalling langchain-0.3.26:
      Successfully uninstalled langchain-0.3.26


In [3]:
!pip install -U langchain-groq

In [4]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [5]:

from langchain_groq import ChatGroq

In [7]:

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq


# 📄 Load the PDF file
pdf_path = "/content/budget_speech.pdf"
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

# 🧩 Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# 📊 Embed and store in Chroma
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="./chroma_db")
retriever = vector_store.as_retriever()

#   Use LLaMA3-70B from Groq
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)

#   Set up RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

#    Main loop
print("\n📄 PDF Q&A Bot (Groq + LLaMA3-70B)")
print("Ask anything from the PDF. Type 'exit' to quit.\n")

while True:
    query = input("🧠 You: ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Bye!")
        break

    result = qa_chain({"query": query})
    answer = result['result'].strip()

    # Handle empty/irrelevant answers
    if not answer or answer.lower() in ["i don't know", "i cannot answer that", ""]:
        print("\n❌ Sorry, I couldn’t find the answer in the document.\n")
    else:
        print("\n🤖 Answer:", answer)

    # Optional: show source info
    print("\n📚 Source(s):")
    for doc in result['source_documents']:
        print(f"• {doc.metadata.get('source', 'Unknown')} | Page: {doc.metadata.get('page', 'N/A')}")
    print("\n" + "-"*60 + "\n")



📄 PDF Q&A Bot (Groq + LLaMA3-70B)
Ask anything from the PDF. Type 'exit' to quit.

🧠 You: the great telugu poet

🤖 Answer: The great Telugu poet mentioned in the text is Gurajada Appa Rao.

📚 Source(s):
• /content/budget_speech.pdf | Page: 5
• /content/budget_speech.pdf | Page: 5
• /content/budget_speech.pdf | Page: 0
• /content/budget_speech.pdf | Page: 0

------------------------------------------------------------

🧠 You: Building Rural Prosperity and Resilience

🤖 Answer: Based on the provided context, Building Rural Prosperity and Resilience is a comprehensive multi-sectoral program that aims to:

1. Enhance agricultural productivity
2. Adopt crop diversification and sustainable agriculture practices
3. Augment post-harvest storage at the panchayat and block level
4. Improve irrigation facilities
5. Facilitate availability of long-term and short-term credit

This program is likely to help 1.7 crore farmers.

Additionally, the program will focus on:

1. Catalyzing enterprise devel

KeyboardInterrupt: Interrupted by user